In [13]:
%reload_ext autoreload
%autoreload 2

import numpy as np
import bloscpack as bp

import sys
import os
sys.path.append(os.path.join(os.environ['REPO_DIR'], 'utilities'))
from utilities2015 import *
from annotation_utilities import *
from registration_utilities import *
from metadata import *
from data_manager import *

import pandas as pd

import time

import matplotlib.pyplot as plt
%matplotlib inline

from vis3d_utilities import *

In [2]:
with open('colors.txt', 'r') as f:
    colors = {l[0]: np.r_[float(l[1]), float(l[2]), float(l[3])] for l in map(lambda x: x.split(), f.readlines())}

In [3]:
train_sample_scheme = 1
global_transform_scheme = 1
local_transform_scheme = 1

In [4]:
stack = 'MD590'

In [5]:
paired_structures = ['5N', '6N', '7N', '7n', 'Amb', 'LC', 'LRt', 'Pn', 'Tz', 'VLL', 'RMC', 'SNC', 'SNR', '3N', '4N',
                    'Sp5I', 'Sp5O', 'Sp5C', 'PBG', '10N', 'VCA', 'VCP', 'DC']
singular_structures = ['AP', '12N', 'RtTg', 'SC', 'IC']
structures = paired_structures + singular_structures

structure_colors = {n: np.random.randint(0, 255, (3,)) for n in structures}
# with open('colors.txt', 'r') as f:
#     colors = {l[0]: np.r_[float(l[1]), float(l[2]), float(l[3])] for l in map(lambda x: x.split(), f.readlines())}

In [11]:
structures_sided = sum([[n] if n in singular_structures else [convert_to_left_name(n), convert_to_right_name(n)]
                        for n in structures], [])

In [18]:
# Render globally transformed atlas structures

for name_s in structures_sided:
# for name_s in ['12N']:

    print name_s
    
    reversed_volume = \
    DataManager.load_transformed_volume(stack_m='atlas_on_MD589', type_m='score', 
                                        stack_f=stack, type_f='score',
                                        downscale=32, 
                                        train_sample_scheme_f=train_sample_scheme,
                                        global_transform_scheme=global_transform_scheme,
                                        local_transform_scheme=local_transform_scheme,
                                        label=name_s, transitive='by')
    
    xmin, xmax, ymin, ymax, zmin, zmax = bbox_3d(reversed_volume)
    vol = reversed_volume[ymin:ymax+1, xmin:xmax+1, zmin:zmax+1]
    
    dense_vol = fill_sparse_score_volume(vol)
    
    t = time.time()

    polydata = volume_to_polydata(dense_vol, origin=(xmin,ymin,zmin), num_simplify_iter=3, smooth=True)

    sys.stderr.write('time: %.2f\n' % (time.time() - t))

    mesh_fn = DataManager.get_transformed_volume_mesh_filepath(stack_m='atlas_on_MD589', type_m='score', 
                                                         stack_f=stack, type_f='score', 
                                                         downscale=32,
                                                        train_sample_scheme_f=train_sample_scheme,
                                                        global_transform_scheme=global_transform_scheme,
                                                        local_transform_scheme=local_transform_scheme,
                                                        label=name_s, transitive='by')     
        
    save_mesh(polydata, mesh_fn)

5N_L


marching cube: 0.02 seconds
compute surface area: 0.00 seconds
mesh_to_polydata: 0.01 seconds
simplify 0 @ 11612: 0.10 seconds
simplify 1 @ 2313: 0.09 seconds
simplify 2 @ 464: 0.02 seconds
time: 0.25


5N_R


marching cube: 0.02 seconds
compute surface area: 0.00 seconds
mesh_to_polydata: 0.01 seconds
simplify 0 @ 12044: 0.10 seconds
simplify 1 @ 2389: 0.14 seconds
simplify 2 @ 480: 0.02 seconds
time: 0.31


6N_L


...

marching cube: 0.36 seconds
compute surface area: 0.03 seconds
mesh_to_polydata: 0.14 seconds
simplify 0 @ 55612: 1.26 seconds
simplify 1 @ 10952: 0.99 seconds
simplify 2 @ 2123: 0.21 seconds
time: 3.01


SC


marching cube: 0.85 seconds
compute surface area: 0.12 seconds
mesh_to_polydata: 0.33 seconds
simplify 0 @ 250365: 3.00 seconds
simplify 1 @ 49111: 4.16 seconds
simplify 2 @ 9772: 0.69 seconds
time: 9.21


IC


marching cube: 0.55 seconds
compute surface area: 0.04 seconds
mesh_to_polydata: 0.18 seconds
simplify 0 @ 140126: 1.31 seconds
simplify 1 @ 27718: 1.56 seconds
simplify 2 @ 5537: 0.34 seconds
time: 4.01


In [19]:
# Load locally transformed atlas structure meshes

reversed_polydata_list = {}

for name_s in structures_sided:
    
    mesh_fn = DataManager.get_transformed_volume_mesh_filepath(stack_m='atlas_on_MD589', type_m='score', 
                                                         stack_f=stack, type_f='score', 
                                                         downscale=32,
                                                        train_sample_scheme_f=train_sample_scheme,
                                                        global_transform_scheme=global_transform_scheme,
                                                        local_transform_scheme=local_transform_scheme,
                                                        label=name_s, transitive='by')     
    
    polydata = load_mesh_stl(mesh_fn, return_polydata_only=True)
    reversed_polydata_list[name_s] = polydata

In [20]:
launch_vtk([actor_mesh(p, wireframe=True, color=structure_colors[convert_name_to_unsided(n)]/255., opacity=.3) 
            for n, p in reversed_polydata_list.iteritems()])

In [22]:
shell_mesh_fn = DataManager.get_shell_mesh_filepath(stack='MD589', downscale=32)
shell_polydata = load_mesh_stl(shell_mesh_fn, return_polydata_only=True)

In [27]:
crop_xmin, crop_xmax, crop_ymin, crop_ymax = metadata_cache['cropbox']['MD589']
print 'crop:', crop_xmin, crop_xmax, crop_ymin, crop_ymax

crop: 569 1053 140 512


In [28]:
tb_vol_xmin, tb_vol_xmax, tb_vol_ymin, tb_vol_ymax, tb_vol_zmin, tb_vol_zmax = \
DataManager.load_volume_bbox(stack='MD589', type='thumbnail', downscale=32)

print 'fixed volume box:', tb_vol_xmin, tb_vol_xmax, tb_vol_ymin, tb_vol_ymax, tb_vol_zmin, tb_vol_zmax

fixed volume box: 0 484 0 372 123 503


In [30]:
launch_vtk([actor_mesh(p, 
                       wireframe=True, 
                       color=structure_colors[convert_name_to_unsided(n)]/255., 
                       opacity=.3,
                      origin=(crop_xmin + tb_vol_xmin, crop_ymin + tb_vol_ymin, tb_vol_zmin))
            for n, p in reversed_polydata_list.iteritems()] + \
           [actor_mesh(shell_polydata, opacity=.1)])